# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [51]:
import pandas as pd

def cargar_datos():
    colspecs = [(0, 10), (10, 40), (40, 50), (50, 55)]
    df = pd.read_fwf('datos.dat', colspecs=colspecs, header=None, names=['fecha', 'producto', 'precio', 'cantidad'])
    df['precio'] = pd.to_numeric(df['precio'], errors='coerce')
    df['cantidad'] = pd.to_numeric(df['cantidad'], errors='coerce')

    return df

datos_df = cargar_datos()

### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [52]:
def calcular_totales(df):
    total_p = (df['precio'] * df['cantidad']).sum()
    total_c = df['cantidad'].sum()
    return total_p, total_c

importe, cantidad = calcular_totales(datos_df)

print(f"Las ventas fueron de: ${importe:.2f} en {cantidad} unidades")

Las ventas fueron de: $392730.00 en 335 unidades


### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [65]:
def unidades_vendidas(df):
    ventas = df.groupby('producto')['cantidad'].sum()
    return ventas

def listar_ventas(ventas):
    for producto, cantidad in ventas.items():
        print(f"Producto: '{producto}'")
        print(f"Unidades vendidas: {cantidad}")
        print()

ventas = unidades_vendidas(datos_df)
listar_ventas(ventas)

Producto: 'Coca Cola'
Unidades vendidas: 57

Producto: 'Mirinda'
Unidades vendidas: 85

Producto: 'Pepsi Cola'
Unidades vendidas: 89

Producto: 'Sprite'
Unidades vendidas: 72

Producto: 'Torasso'
Unidades vendidas: 32



###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [66]:
def precios_promedio(df):
    acumulado = df.groupby('producto', group_keys=False).apply(lambda x: pd.Series({
        'total_precio': (x['precio'] * x['cantidad']).sum(),
        'total_cantidad': x['cantidad'].sum()
    }))
    
    acumulado['precio_promedio'] = acumulado['total_precio'] / acumulado['total_cantidad']
    return acumulado['precio_promedio']

def listar_precios_promedio(precios_promedio):
    for producto, promedio in precios_promedio.items():
        print(f"Producto: '{producto}'")
        print(f"Precio promedio: ${promedio:.2f}")
        print()

precios_promedio = precios_promedio(datos_df)
listar_precios_promedio(precios_promedio)

Producto: 'Coca Cola'
Precio promedio: $1066.32

Producto: 'Mirinda'
Precio promedio: $1542.12

Producto: 'Pepsi Cola'
Precio promedio: $1241.69

Producto: 'Sprite'
Precio promedio: $847.78

Producto: 'Torasso'
Precio promedio: $916.25



C:\Users\Estudiante\AppData\Local\Temp\ipykernel_3096\1804145980.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  acumulado = df.groupby('producto', group_keys=False).apply(lambda x: pd.Series({


### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [67]:
def ranking_productos(df, top=3):
    ventas_productos = df.groupby('producto')['cantidad'].sum()
    ranking = ventas_productos.nlargest(top)
    return ranking

def listar_ranking(ranking):
    print(f"Top {len(ranking)} productos más vendidos:")
    for i, (producto, cantidad) in enumerate(ranking.items(), start=1):
        print(f"{i}. {producto}: {cantidad} unidades")

ranking = ranking_productos(datos_df)
listar_ranking(ranking)

Top 3 productos más vendidos:
1. Pepsi Cola: 89 unidades
2. Mirinda: 85 unidades
3. Sprite: 72 unidades


### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [61]:
def ventas_por_mes(df):
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['mes'] = df['fecha'].dt.month.astype(str).str.zfill(2)  # Formatear mes con ceros a la izquierda
    
    ventas = df.groupby(['producto', 'mes'])['cantidad'].sum().unstack(fill_value=0)
    return ventas

def listar_ventas_mensuales(ventas):
    for producto, meses in ventas.iterrows():
        print(f"Producto: '{producto}'")
        for mes, total in meses.items():
            print(f"  Mes: {mes} -> Unidades vendidas: {total}")
        print()
ventas = ventas_por_mes(datos_df)
listar_ventas_mensuales(ventas)

Producto: 'Coca Cola'
  Mes: 08 -> Unidades vendidas: 0
  Mes: 09 -> Unidades vendidas: 57

Producto: 'Mirinda'
  Mes: 08 -> Unidades vendidas: 27
  Mes: 09 -> Unidades vendidas: 58

Producto: 'Pepsi Cola'
  Mes: 08 -> Unidades vendidas: 10
  Mes: 09 -> Unidades vendidas: 79

Producto: 'Sprite'
  Mes: 08 -> Unidades vendidas: 0
  Mes: 09 -> Unidades vendidas: 72

Producto: 'Torasso'
  Mes: 08 -> Unidades vendidas: 8
  Mes: 09 -> Unidades vendidas: 24



### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [63]:
def resumen_ventas(df):
    resumen = df.groupby('producto').agg(
        total_cantidad=('cantidad', 'sum'),
        total_importe=('precio', lambda x: (x * df.loc[x.index, 'cantidad']).sum()),
        num_registros=('cantidad', 'count')
    )
    resumen['precio_promedio'] = resumen['total_importe'] / resumen['total_cantidad']
    return resumen

def informe_ventas(resumen):
    informe = resumen.sort_index()
    
    print(f"{'Producto':<15} {'Precio Promedio':<20} {'Cantidad':<10} {'Importe Total':<15}")
    print(f'-' * 60)
    print()
    for producto, datos in informe.iterrows():
        print(f"-{producto:<15} ${datos['precio_promedio']:<22,.2f} {datos['total_cantidad']:<8} ${datos['total_importe']:,.2f}")
        print()
    print(f"-" * 60)

resumen = resumen_ventas(datos_df)
informe_ventas(resumen)

Producto        Precio Promedio      Cantidad   Importe Total  
------------------------------------------------------------

-Coca Cola       $1,066.32               57.0     $60,780.00

-Mirinda         $1,542.12               85.0     $131,080.00

-Pepsi Cola      $1,241.69               89.0     $110,510.00

-Sprite          $847.78                 72.0     $61,040.00

-Torasso         $916.25                 32.0     $29,320.00

------------------------------------------------------------


## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [68]:
import numpy as np

puntos = np.array([(0, 0), (1, 8), (2, 12), (3, 12), (5, 0)])

def f_cuadratica(a, b, c, x):
    return a * x ** 2 + b * x + c

x_vals = puntos[:, 0]
y_real_vals = puntos[:, 1]

for a in range(-10, 11):
    for b in range(-10, 11):
        for c in range(-10, 11):
            y_calculado = f_cuadratica(a, b, c, x_vals)
            if np.array_equal(y_calculado, y_real_vals):
                print(f"Valores encontrados: a = {a}, b = {b}, c = {c}")

Valores encontrados: a = -2, b = 10, c = 0


### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [83]:
import numpy as np
from numpy.random import randint, uniform

def f(x, coeficientes):
    a, b, c = coeficientes
    return a * x ** 2 + b * x + c

def error(y, y_pred):
    return np.sum((y - y_pred) ** 2)

def buscar_coeficientes():
    X = np.array([0, 1, 2, 3, 5])
    Y = np.array([0, 8, 12, 11, 1])
    coeficientes = randint(-10, 10, 3)
    y_pred = f(X, coeficientes)
    error_actual = error(Y, y_pred)

    aprendizaje = 0.1

    while error_actual > 1:
        nuevos_coeficientes = coeficientes + uniform(-aprendizaje, aprendizaje, 3)

        y_pred_nueva = f(X, nuevos_coeficientes)
        nuevo_error = error(Y, y_pred_nueva)

        if nuevo_error < error_actual:
            coeficientes = nuevos_coeficientes
            error_actual = nuevo_error
    return coeficientes

coeficientes = buscar_coeficientes()
print("Coeficientes encontrados:", coeficientes)

Coeficientes encontrados: [-1.79401811  9.02219284  0.60383085]
